In [35]:
import schedule
import time
import datetime
import yfinance as yf
import robin_stocks as r
import logging

# Configure the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')
logger = logging.getLogger()

# Define the market open and close times
market_open_time = datetime.time(hour=6, minute=30)
market_close_time = datetime.time(hour=13)

In [ ]:
# Set Robinhood API credentials
robin_user = 'raghunadh.chilukamari@gmail.com'
robin_pass = 'Chiluk@m@ri1991'

Login to Robinhood
r.login(username=robin_user,
         password=robin_pass,
         expiresIn=86400,
         by_sms=True)

In [36]:
# Define the stock symbols and quantities as inputs
stock_symbols = ['SIRI']
quantities = [1]

In [57]:
def day_sniper_strategy(symbol, quantity):
    global buy_order_placed
    global stop_loss_order_placed
    
    current_time = datetime.datetime.now().time()
    logger.info(f"current time is : {current_time}")
    
    
    try:
        logger.info(f"Running day sniper strategy for symbol: {symbol}")

        # Get historical data using yfinance or provide sample data for testing
        data = yf.download(symbol, period='1d', interval='15m')
        
        # Extract the close price and low price of the first 15-minute candle
        first_candle = data.iloc[0]
        close_price = first_candle['Close']
        low_price = first_candle['Low']
        
        if market_open_time <= current_time <= (market_open_time + datetime.timedelta(minutes=15)) and not buy_order_placed:

            logger.info(f"Placing buy order for symbol: {symbol}, quantity: {quantity}, price: {close_price}")
            
            # Place the order with the close price of the first 15-minute candle          
            # r.order_buy_limit(symbol, quantity, close_price)
            buy_order_placed = True

            logger.info(f"Setting stop loss order for symbol: {symbol}, quantity: {quantity}, stop price: {low_price}")

            # Set the stop loss at the low price of the first 15-minute candle
            # r.order_sell_stop_loss(symbol, quantity, low_price)
            
            stop_loss_order_placed = True

        logger.info("Checking if stop loss order got executed")
        stop_loss_order = None
        
        if stop_loss_order_placed:

            # Check if the stop loss order is still open
            open_orders = []
            # open_orders = r.orders.get_open_stock_orders()
            # stop_loss_order_status = r.orders.get_stock_order_info(stop_loss_order['id'])['state']
            # stop_loss_order = next((order for order in open_orders if order['side'] == 'sell' and order['stop_price'] == low_price), None)
            
            for order in open_orders:
                if order['symbol'] == symbol and order['stop_price'] == low_price:
                    stop_loss_order = order
                    break

        if stop_loss_order is None:
            logger.info(f"Stop loss order for symbol: {symbol} got executed")
        else:
            logger.info(f"Stop loss order for symbol: {symbol} is still open")

            # Check if it is the last candle of the day
            last_candle = data.iloc[-1]
            print(last_candle.name.time())

            if last_candle.name.time() >= (market_close_time - datetime.timedelta(minutes=15):
                logger.info(f"Last candle of the day for symbol: {symbol}")

                # The stop loss order didn't get executed, place a market sell order
                logger.info(f"Placing market sell order for symbol: {symbol}, quantity: {quantity}")
                # r.order_sell_market(symbol, quantity)
                logger.info(f"Strategy execution completed for symbol: {symbol}")
                # schedule.cancel_job(job)

            else:
                logger.info("Regular market hours, waiting until close to check stop loss")       

    except Exception as e:
        logger.error(f"Error occurred while running strategy for symbol: {symbol}")
        logger.error(str(e))

In [58]:
def run_day_sniper_every_15min(symbol, quantity):
    # Get current time and check if it's within the desired time range (6:30 AM to 1:00 PM MST)
    current_time = datetime.datetime.now().time()

    if market_open_time <= current_time <= market_close_time:
        day_sniper_strategy(symbol, quantity)
        

In [59]:
# Schedule the strategy to run at the market start time for each stock symbol
for symbol, quantity in zip(stock_symbols, quantities):
    schedule.every().day.at(str(market_open_time)).do(day_sniper_strategy, symbol=symbol, quantity=quantity)    

In [60]:

# Schedule the strategy to run every 15 minutes for each stock symbol
for symbol, quantity in zip(stock_symbols, quantities):
    schedule.every(15).minutes.until(market_close_time).do(run_day_sniper_every_15min, symbol=symbol, quantity=quantity)

In [61]:
# Keep the program running and execute scheduled tasks
while True:
    current_time = datetime.datetime.now().time()
    if current_time > market_close_time:
        logger.info("After market hours, exiting loop")  
        break
    
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 

In [62]:
all_jobs = schedule.get_jobs()
all_jobs

[Every 1 day at 00:30:00 do day_sniper_strategy(symbol='SIRI', quantity=1) (last run: 2023-07-02 00:45:18, next run: 2023-07-03 00:30:00),
 Every 15 minutes do run_day_sniper_every_15min(symbol='SIRI', quantity=1) (last run: 2023-07-02 00:45:19, next run: 2023-07-02 01:00:19),
 Every 1 day at 00:30:00 do day_sniper_strategy(symbol='SIRI', quantity=1) (last run: [never], next run: 2023-07-03 00:30:00),
 Every 15 minutes do run_day_sniper_every_15min(symbol='SIRI', quantity=1) (last run: [never], next run: 2023-07-02 01:10:02)]

NameError: name 'job' is not defined